In [1]:
#Import required modules

from tensorflow.keras.models import model_from_json
import cv2
import numpy as np

In [2]:
# Load model from .json file created after training
with open('model.json', "r") as json_file:
            loaded_model_json = json_file.read()
            model = model_from_json(loaded_model_json)
            
model.load_weights('new_model/model.weights.h5')   # Load trained weights


EMOTIONS_LIST = ["Angry", "Happy", "Sad"]

def predict_emotion(img):  # Predict Emotions
        preds = model.predict(img)
        return EMOTIONS_LIST[np.argmax(preds)]

facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX

C:\Users\burci\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 54 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 13 objects could not be loaded. Example error message for object <Conv2D name=conv2d, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(3, 3, 1, 64), Received: value.shape=(3, 3, 1, 32). Target variable: <KerasVariable shape=(3, 3, 1, 64), dtype=float32, path=conv2d/kernel>

List of objects that could not be loaded:
[<Conv2D name=conv2d, built=True>, <BatchNormalization name=batch_normalization, built=True>, <Conv2D name=conv2d_1, built=True>, <BatchNormalization name=batch_normalization_1, built=True>, <Conv2D name=conv2d_2, built=True>, <BatchNormalization name=batch_normalization_2, built=True>, <Conv2D name=conv2d_3, built=True>, <BatchNormalization name=batch_normalization_3, built=True>, <Dense name=dense, built=True>, <BatchNormalization name=batch_normalization_4, built=True>, <Dense name=dense_1, built=True>, <BatchNormalization name=batch_normalization_5, built=True>,

In [3]:
video = cv2.VideoCapture(0)

while True:
    _, fr = video.read()
    gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    faces = facec.detectMultiScale(gray_fr, 1.3, 5)

    for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            roi = roi[np.newaxis, :, :, np.newaxis]
            pred = predict_emotion(roi)

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.imshow('img',fr)
    k = cv2.waitKey(1) & 0xff
    if k==ord("q"):
        break

video.release()
cv2.destroyAllWindows()

NameError: name 'facec' is not defined

In [4]:
import cv2
import numpy as np
import requests
import base64
import json

# video capture ı başlat
video = cv2.VideoCapture(0)

# frameleri Flask servera gönderen ve tahmini getiren fonksiyon
def send_frame_to_flask(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    frame_base64 = base64.b64encode(buffer).decode('utf-8')
    payload = json.dumps({'frame': frame_base64})
    response = requests.post('http://192.168.1.91:5000/predict_emotion', data=payload, headers={'Content-Type': 'application/json'})
    if response.status_code == 200:
        emotion= response.json().get('emotion', 'Unknown')
        print(f"Received emotion: {emotion}")
        return emotion
    print(f"Error: {response.status_code}")
    return 'Error'

while True:
    ret, frame = video.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray_frame, 1.3, 5)

    for (x, y, w, h) in faces:
        face_crop = gray_frame[y:y+h, x:x+w]
        face_crop = cv2.resize(face_crop, (48, 48))
        
        # Send the cropped face frame to Flask server for emotion prediction
        emotion = send_frame_to_flask(face_crop)

        # Display the prediction result on the video frame
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    cv2.imshow('Emotion Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

# ENTEGRE MODELE KARAR VERILDIGI ICIN BUNU GELISTIRMEDIM AMA
# GELIŞTIRILEBILIR, ÇALIŞMIYORDU EN SON...


ConnectTimeout: HTTPConnectionPool(host='192.168.1.91', port=5000): Max retries exceeded with url: /predict_emotion (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000198D0C43AA0>, 'Connection to 192.168.1.91 timed out. (connect timeout=None)'))

In [17]:
# working but model is silly (belirtilen pathdeki keras modelinde, mutlu nerdeyse hiç algılanmıyor)
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the Keras model
model = load_model('trained_emotion_model2.keras')

# Define emotion labels
emotion_labels = ["Happy", "Sad", "Angry"]  # Modify according to your labels

def preprocess_frame(frame):
    # Preprocess the frame for the model (resize, normalize, etc.)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (224,224))  
    normalized = resized / 255.0
    return np.expand_dims(normalized, axis=(0, -1))

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    preprocessed_frame = preprocess_frame(frame)

    # Predict emotion
    predictions = model.predict(preprocessed_frame)
    emotion = emotion_labels[np.argmax(predictions)]

    # Display emotion
    cv2.putText(frame, f'Emotion: {emotion}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show frame
    cv2.imshow('Webcam', frame)

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

In [16]:
# mutluyu iyi algılamadı (şu anki pathde olan tflite modelinde)

import cv2
import numpy as np
import tensorflow as tf

# TFLite modeli yükle ve tensorlerini al
interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

# input ve output tensorlerini getir
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Duygu labellarını ayarla
emotion_labels = ["Happy", "Sad", "Angry"]  # Modify according to your labels

def preprocess_frame(frame):
    # model için Frame üzerinde Preprocess işlemi  (resize, normalize, etc.)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (224,224))  # input sizeı 224 x 224 olarak varsay
    normalized = resized / 255.0
    return np.expand_dims(normalized, axis=(0, -1)).astype(np.float32)

# webcami aç
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    preprocessed_frame = preprocess_frame(frame)

    # Set tensor for the model
    interpreter.set_tensor(input_details[0]['index'], preprocessed_frame)

    # Çıkarımı runla
    interpreter.invoke()

    # Get output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    emotion = emotion_labels[np.argmax(output_data)]

    cv2.putText(frame, f'Emotion: {emotion}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Webcam', frame)     # Show frame

    if cv2.waitKey(1) & 0xFF == ord('q'):    # Break on 'q' key
        break

cap.release()
cv2.destroyAllWindows()
